### С помощью подхода MapReduce посчитайте:
 - среднюю погоду по каждому городу за представленный период;
 - среднюю температуру по федеральному округу;
 - разницу между средней температурой города и федерального округа, к которому относится этот город.


In [38]:
import pandas as pd
import csv

In [39]:
df_city = pd.read_csv('src/city_weather.csv')
df_district = pd.read_csv('src/district_to_city.csv')
df_city.head()

,city,date,day_temperatre,night_temperature
0,Оренбург,2020-03-30,15.6,-2.0
1,Липецк,2020-03-21,6.6,-3.0
2,Челябинск,2020-03-23,-2.7,-7.1
3,Оренбург,2020-03-22,5.8,-6.0
4,Томск,2020-03-29,5.7,-5.3


In [40]:
# city mapper
with open('temp/mapped_city.csv', 'w',  encoding='utf-8', newline='') as mapped_city:
    mapped_city_writer = csv.writer(mapped_city)
    mapped_city_writer.writerow(['city','district','daily_temp',])
    with open('src/city_weather.csv', 'r',  encoding='utf-8') as city_weather:
            for i,city_line in enumerate(city_weather):
                if i == 0:
                    continue
                city_line = city_line.strip().split(',')
                city = city_line[0]
                temperature = round((float(city_line[2]) + float(city_line[3]))/2,1)
                # print(city_line)
                if city in df_district['city'].values:
                    city_district = df_district.loc[df_district['city'] == city, 'federal_district'].iloc[0]
                    mapped_city_writer.writerow([city, city_district, temperature])

In [41]:
df_new = pd.read_csv('temp/mapped_city.csv')
df_new.head()


,city,district,daily_temp
0,Оренбург,Приволжский ФО,6.8
1,Липецк,Центральный ФО,1.8
2,Челябинск,Уральский ФО,-4.9
3,Оренбург,Приволжский ФО,-0.1
4,Томск,Сибирский ФО,0.2


In [42]:
# этап filter
def multifunctioanl_filer(input_filepath:str,column:str, another_column:str, output_filepath:str) -> None:
    with open(input_filepath, 'r', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)  # Читаем заголовок
        data = list(reader)  # Читаем остальные данные

    column_index = header.index(column)
    another_index = header.index(another_column)
    # Сортируем данные по указанной колонке
    sorted_data = sorted(data, key=lambda row: (row[column_index], row[another_index]))

    # Записываем отсортированные данные в новый файл
    with open(output_filepath, 'w', encoding='utf-8', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(header)  # Записываем заголовок
        writer.writerows(sorted_data)

In [43]:
multifunctioanl_filer(input_filepath='temp/mapped_city.csv', column='district', another_column='city', output_filepath='temp/sorted_city.csv')
df_new = pd.read_csv('temp/sorted_city.csv')
df_new.head(10)


,city,district,daily_temp
0,Анадырь,Дальневосточный ФО,-10.4
1,Анадырь,Дальневосточный ФО,-5.8
2,Анадырь,Дальневосточный ФО,-23.8
3,Анадырь,Дальневосточный ФО,-22.8
4,Анадырь,Дальневосточный ФО,-3.5
5,Анадырь,Дальневосточный ФО,-21.9
6,Анадырь,Дальневосточный ФО,-4.7
7,Анадырь,Дальневосточный ФО,-28.0
8,Анадырь,Дальневосточный ФО,-16.0
9,Анадырь,Дальневосточный ФО,-9.9


In [44]:
# этап reduce
def city_reducer(input_filepath:str,column:str,output_filepath) -> None:
    previous_city = None
    previous_district = None
    city_days = 0
    district_amount_cities = 0
    with open('temp/sorted_city.csv', 'r', encoding='utf-8') as file_input:
        with open('result.csv', 'w', newline='', encoding='utf-8') as file_output:
            district_cities = []
            csv.writer(file_output).writerow(['city', 'avg_city_temp', 'district', 'city_temp_deviation' ,'avg_district_temp'])

            for i,line in enumerate(file_input):
                if i == 0:
                    continue
                line = line.strip().split(',')
                if previous_city:
                    if previous_city == line[0]:
                        city_temp += float(line[2])
                        city_days += 1
                    else:
                        city_temp = round(city_temp / city_days,1)
                        district_cities.append([previous_city, city_temp, line[1]])
                        district_temp += city_temp
                        district_amount_cities += 1
                        city_temp = 0
                        city_days = 0
                        previous_city = line[0]
                else:
                    city_temp = 0
                    previous_city = line[0]

                if previous_district:
                    if previous_district != line[1]:
                        district_temp = round(district_temp / district_amount_cities, 1)
                        for city in district_cities:
                            city.extend([round(city[1]-district_temp,1), district_temp])
                            csv.writer(file_output).writerow(city)
                        district_temp = 0
                        district_amount_cities = 0
                        district_cities = []
                        previous_district = line[1]
                else:
                    district_temp = 0
                    district_amount_cities = 0
                    previous_district = line[1]


In [45]:
city_reducer(input_filepath='temp/sorted_city.csv', column='city', output_filepath='result.csv')

In [49]:
df_result = pd.read_csv('result.csv')
df_result.sample(20)

,city,avg_city_temp,district,city_temp_deviation,avg_district_temp
17,Самара,2.3,Приволжский ФО,0.8,1.5
0,Анадырь,-12.7,Дальневосточный ФО,-6.3,-6.4
63,Тамбов,4.6,Центральный ФО,0.9,3.7
35,Ставрополь,7.0,Сибирский ФО,-0.3,7.3
44,Улан-Удэ,-4.9,Сибирский ФО,-0.7,-4.2
30,Сыктывкар,-1.9,Северо-Кавказский ФО,-1.1,-0.8
29,Санкт-Петербург,1.6,Северо-Западный ФО,2.4,-0.8
20,Чебоксары,2.0,Северо-Западный ФО,0.5,1.5
41,Новосибирск,-3.9,Сибирский ФО,0.3,-4.2
8,Якутск,-17.0,Приволжский ФО,-10.6,-6.4
